In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xlsx',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

# Data from each sheet can be accessed via key
keyList = list(dfs.keys())

# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Change as China for coordinate search
    dfs[key]=dfs[key].replace({'Country/Region':'Mainland China'}, 'China')
    dfs[key]=dfs[key].replace({'Province/State':'Queensland'}, 'Brisbane')
    dfs[key]=dfs[key].replace({'Province/State':'New South Wales'}, 'Sydney')
    dfs[key]=dfs[key].replace({'Province/State':'Victoria'}, 'Melbourne')
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    # Convert time as Australian eastern daylight time
    dfs[key]['Date_last_updated_AEDT'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]
    dfs[key]['Date_last_updated_AEDT'] = dfs[key]['Date_last_updated_AEDT'] + timedelta(hours=16)
    
# Check 
dfs[keyList[0]].head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,City,Date_last_updated_AEDT
0,Hubei,China,02/27/2020 19:00,65914,2682,26403,NaN,2020-02-28 11:00:00
1,Guangdong,China,02/27/2020 19:00,1347,7,890,NaN,2020-02-28 11:00:00
2,Zhejiang,China,02/27/2020 19:00,1205,1,932,NaN,2020-02-28 11:00:00
3,Shandong,China,02/27/2020 19:00,756,6,387,NaN,2020-02-28 11:00:00
4,Henan,China,02/27/2020 19:00,1272,20,1068,NaN,2020-02-28 11:00:00


In [3]:
# Import data with coordinates (coordinates were called seperately in "Updated_coordinates")
dfs[keyList[0]]=pd.read_csv('{}_data.csv'.format(keyList[0]))

In [4]:
dfs[keyList[1]]['Deaths'].sum()

2814

In [5]:
# Save numbers into variables to use in the app
confirmedCases=dfs[keyList[0]]['Confirmed'].sum()
deathsCases=dfs[keyList[0]]['Deaths'].sum()
recoveredCases=dfs[keyList[0]]['Recovered'].sum()

In [6]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Confirmed'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Confirmed'][0])
    OtherList.append(dfTpm['Confirmed'][1:].sum())
    
df_confirmed = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList})    

df_confirmed['date_day']=[d.date() for d in df_confirmed['Date']]
df_confirmed=df_confirmed.groupby(by=df_confirmed['date_day'], sort=False).transform(max).drop_duplicates(['Date'])
df_confirmed['Total']=df_confirmed['Mainland China']+df_confirmed['Other locations']
df_confirmed=df_confirmed.reset_index(drop=True)
df_confirmed

,Date,Mainland China,Other locations,Total
0,2020-02-28 11:00:00,78815,4281,83096
1,2020-02-27 22:00:00,78497,4010,82507
2,2020-02-26 22:00:00,78064,3157,81221
3,2020-02-25 22:00:00,77660,2667,80327
4,2020-02-24 22:00:00,77150,2360,79510
5,2020-02-23 22:00:00,76936,1948,78884
6,2020-02-22 22:00:00,76291,1634,77925
7,2020-02-21 22:00:00,75247,1337,76584
8,2020-02-20 22:00:00,74578,1183,75761
9,2020-02-19 22:00:00,74184,1106,75290


In [7]:
# Construct new dataframe for 24-hour window case difference
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Confirmed'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Confirmed'][0])
    OtherList.append(dfTpm['Confirmed'][1:].sum())
    
df_confirmed_diff = pd.DataFrame({'Date':DateList,
                                  'Mainland China':ChinaList,
                                  'Other locations':OtherList}) 
df_confirmed_diff['Total']=df_confirmed_diff['Mainland China']+df_confirmed_diff['Other locations']

# Calculate differenec in a 24-hour window
for index, _ in df_confirmed_diff.iterrows():
    # Calculate the time differnece in hour
    diff=(df_confirmed_diff['Date'][0] - df_confirmed_diff['Date'][index]).total_seconds()/3600
    # find out the latest time after 24-hour
    if diff >= 24:
        break
plusConfirmedNum = df_confirmed_diff['Total'][0] - df_confirmed_diff['Total'][index]
plusPercentNum1 = (df_confirmed_diff['Total'][0] - df_confirmed_diff['Total'][index])/df_confirmed_diff['Total'][index]

In [8]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Recovered'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Recovered':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Recovered', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Recovered'][0])
    OtherList.append(dfTpm['Recovered'][1:].sum())
    
df_recovered = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList}) 

df_recovered['date_day']=[d.date() for d in df_recovered['Date']]
df_recovered=df_recovered.groupby(by=df_recovered['date_day'], sort=False).transform(max).drop_duplicates(['Date'])
df_recovered['Total']=df_recovered['Mainland China']+df_recovered['Other locations']
df_recovered=df_recovered.reset_index(drop=True)
df_recovered

,Date,Mainland China,Other locations,Total
0,2020-02-28 11:00:00,35899,407,36306
1,2020-02-27 22:00:00,32794,349,33143
2,2020-02-26 22:00:00,29998,283,30281
3,2020-02-25 22:00:00,27551,272,27823
4,2020-02-24 22:00:00,24915,238,25153
5,2020-02-23 22:00:00,23101,225,23326
6,2020-02-22 22:00:00,21018,215,21233
7,2020-02-21 22:00:00,18607,198,18805
8,2020-02-20 22:00:00,16648,188,16836
9,2020-02-19 22:00:00,14755,177,14932


In [9]:
# Construct new dataframe for 24-hour window case difference
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Recovered'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Recovered':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Recovered', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Recovered'][0])
    OtherList.append(dfTpm['Recovered'][1:].sum())
    
df_recovered_diff = pd.DataFrame({'Date':DateList,
                                  'Mainland China':ChinaList,
                                  'Other locations':OtherList}) 
df_recovered_diff['Total']=df_recovered_diff['Mainland China']+df_confirmed_diff['Other locations']

# Calculate differenec in a 24-hour window
for index, _ in df_recovered_diff.iterrows():
    # Calculate the time differnece in hour
    diff=(df_recovered_diff['Date'][0] - df_recovered_diff['Date'][index]).total_seconds()/3600
    # find out the latest time after 24-hour
    if diff >= 24:
        break
plusRecoveredNum = df_recovered_diff['Total'][0] - df_recovered_diff['Total'][index]
plusPercentNum2 = (df_recovered_diff['Total'][0] - df_recovered_diff['Total'][index])/df_recovered_diff['Total'][index]

In [10]:
plusPercentNum2

0.20342638073559363

In [11]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Deaths'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Deaths':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Deaths', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Deaths'][0])
    OtherList.append(dfTpm['Deaths'][1:].sum())
    
df_deaths = pd.DataFrame({'Date':DateList,
                          'Mainland China':ChinaList,
                          'Other locations':OtherList})  

df_deaths['date_day']=[d.date() for d in df_deaths['Date']]
df_deaths=df_deaths.groupby(by='date_day', sort=False).transform(max).drop_duplicates(['Date'])
df_deaths['Total']=df_deaths['Mainland China']+df_deaths['Other locations']
df_deaths=df_deaths.reset_index(drop=True)
df_deaths

,Date,Mainland China,Other locations,Total
0,2020-02-28 11:00:00,2787,70,2857
1,2020-02-27 22:00:00,2744,65,2809
2,2020-02-26 22:00:00,2715,54,2769
3,2020-02-25 22:00:00,2663,43,2706
4,2020-02-24 22:00:00,2592,33,2625
5,2020-02-23 22:00:00,2442,23,2465
6,2020-02-22 22:00:00,2345,17,2362
7,2020-02-21 22:00:00,2236,13,2249
8,2020-02-20 22:00:00,2118,11,2129
9,2020-02-19 22:00:00,2006,6,2012


In [12]:
# Construct new dataframe for 24-hour window case difference
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Deaths'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Deaths':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Deaths', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated_AEDT'][0])
    ChinaList.append(dfTpm['Deaths'][0])
    OtherList.append(dfTpm['Deaths'][1:].sum())
    
df_deaths_diff = pd.DataFrame({'Date':DateList,
                                  'Mainland China':ChinaList,
                                  'Other locations':OtherList}) 
df_deaths_diff['Total']=df_deaths_diff['Mainland China']+df_deaths_diff['Other locations']

# Calculate differenec in a 24-hour window
for index, _ in df_deaths_diff.iterrows():
    # Calculate the time differnece in hour
    diff=(df_deaths_diff['Date'][0] - df_deaths_diff['Date'][index]).total_seconds()/3600
    # find out the latest time after 24-hour
    if diff >= 24:
        break
plusDeathNum = df_deaths_diff['Total'][0] - df_deaths_diff['Total'][index]
plusPercentNum3 = (df_deaths_diff['Total'][0] - df_deaths_diff['Total'][index])/df_deaths_diff['Total'][index]

In [13]:
plusPercentNum3

0.031035727174305305

In [14]:
# Generate sum values for Country/Region level
dfCase = dfs[keyList[0]].groupby(by='Country/Region', sort=False).sum().reset_index()
dfCase = dfCase.sort_values(by=['Confirmed'], ascending=False).reset_index(drop=True)
# As lat and lon also underwent sum(), which is not desired, remove from this table.
dfCase = dfCase.drop(columns=['lat','lon'])
dfCase.head()

,Country/Region,Confirmed,Deaths,Recovered
0,China,78815,2787,35899
1,South Korea,1766,13,24
2,Japan,912,8,42
3,Italy,655,17,45
4,Iran,245,26,54


In [15]:
# Grep lat and lon by the first instance to represent its Country/Region
dfGPS = dfs[keyList[0]].groupby(by=['Country/Region'], sort=False).first().reset_index()
dfGPS = dfGPS[['Country/Region','lat','lon']]
dfGPS.head()

,Country/Region,lat,lon
0,China,31.151725,112.878322
1,Hong Kong,22.279328,114.162813
2,Taiwan,23.973937,120.982018
3,Macau,22.175761,113.551414
4,South Korea,35.871300,128.601800


In [16]:
# Merge two dataframes
dfSum = pd.merge(dfCase, dfGPS, how='inner', on='Country/Region')
dfSum = dfSum.replace({'Country/Region':'China'}, 'Mainland China')
dfSum['Remaining'] = dfSum['Confirmed'] - dfSum['Recovered'] - dfSum['Deaths']
dfSum = dfSum[['Country/Region','Remaining','Confirmed','Recovered','Deaths','lat','lon']]
dfSum.head()

,Country/Region,Remaining,Confirmed,Recovered,Deaths,lat,lon
0,Mainland China,40129,78815,35899,2787,31.151725,112.878322
1,South Korea,1729,1766,24,13,35.871300,128.601800
2,Japan,862,912,42,8,35.444991,139.636768
3,Italy,593,655,45,17,45.620900,9.768893
4,Iran,165,245,54,26,32.647531,54.564352


In [17]:
# Save numbers into variables to use in the app
latestDate=datetime.strftime(df_confirmed['Date'][0], '%b %d, %Y %H:%M AEDT')
secondLastDate=datetime.strftime(df_confirmed['Date'][1], '%b %d')
daysOutbreak=(df_confirmed['Date'][0] - datetime.strptime('12/31/2019', '%m/%d/%Y')).days

In [18]:
latestDate

'Feb 28, 2020 11:00 AEDT'

In [32]:
# Line plot for confirmed cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_confirmed['Mainland China'].max()+1000, 10000))

# Create empty figure canvas
fig_confirmed = go.Figure()
# Add trace to the figure
fig_confirmed.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Mainland China'],
                                   mode='lines+markers',
                                   line_shape='spline',
                                   name='Mainland China',
                                   line=dict(color='#921113', width=3),
                                   marker=dict(size=6, color='#f4f4f2',
                                               line=dict(width=1,color='#921113')),
                                   text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_confirmed['Date']],
                                   hovertext=['Mainland China confirmed<br>{:,d} cases<br>'.format(i) for i in df_confirmed['Mainland China']],
                                   hovertemplate='<b>%{text}</b><br></br>'+
                                                 '%{hovertext}'+
                                                 '<extra></extra>'))
fig_confirmed.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Other locations'],
                                   mode='lines+markers',
                                   line_shape='spline',
                                   name='Other Region',
                                   line=dict(color='#eb5254', width=3),
                                   marker=dict(size=6, color='#f4f4f2',
                                               line=dict(width=1,color='#eb5254')),
                                   text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_confirmed['Date']],
                                   hovertext=['Other locations confirmed<br>{:,d} cases<br>'.format(i) for i in df_confirmed['Other locations']],
                                   hovertemplate='<b>%{text}</b><br></br>'+
                                                 '%{hovertext}'+
                                                 '<extra></extra>'))
# Customise layout
fig_confirmed.update_layout(
    #title=dict(
    #    text="<b>Confirmed Cases Timeline<b>",
    #    y=0.96, x=0.5, xanchor='center', yanchor='top',
    #    font=dict(size=20, color="#292929", family="Playfair Display")
    #),
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=["{:.0f}k".format(i/1000) for i in tickList]
    ),
#    yaxis_title="Total Confirmed Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
#    legend=dict(x=.35, y=-.05),
    plot_bgcolor='#f4f4f2',
    paper_bgcolor='#cbd2d3',
    font=dict(color='#292929')
)

fig_confirmed.show()

In [33]:
# Line plot for both recovered and death cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_recovered['Mainland China'].max()+1000, 4000))

# Create empty figure canvas
fig_combine = go.Figure()
# Add trace to the figure
fig_combine.add_trace(go.Scatter(x=df_recovered['Date'], y=df_recovered['Total'],
                                 mode='lines+markers',
                                 line_shape='spline',
                                 name='Total Recovered Cases',
                                 line=dict(color='#168038', width=3),
                                 marker=dict(size=6, color='#f4f4f2',
                                               line=dict(width=1,color='#168038')),
                                 text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_recovered['Date']],
                                 hovertext=['Total recovered<br>{:,d} cases<br>'.format(i) for i in df_recovered['Total']],
                                 hovertemplate='<b>%{text}</b><br></br>'+
                                                 '%{hovertext}'+
                                                 '<extra></extra>'))
fig_combine.add_trace(go.Scatter(x=df_deaths['Date'], y=df_deaths['Total'],
                                mode='lines+markers',
                                 line_shape='spline',
                                name='Total Death Cases',
                                line=dict(color='#626262', width=3),
                                marker=dict(size=6, color='#f4f4f2',
                                            line=dict(width=1,color='#626262')),
                                text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_deaths['Date']],
                                hovertext=['Total death<br>{:,d} cases<br>'.format(i) for i in df_deaths['Total']],
                                hovertemplate='<b>%{text}</b><br></br>'+
                                              '%{hovertext}'+
                                              '<extra></extra>'))
# Customise layout
fig_combine.update_layout(
    #title=dict(
    #    text="<b>Recovered Cases Timeline<b>",
    #    y=0.96, x=0.5, xanchor='center', yanchor='top',
    #    font=dict(size=20, color="#292929", family="Playfair Display")
    #),
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=["{:.0f}k".format(i/1000) for i in tickList]
    ),
#    yaxis_title="Total Recovered Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
#    legend=dict(x=.35, y=-.05),
    plot_bgcolor='#f4f4f2',
    paper_bgcolor='#cbd2d3',
    font=dict(color='#292929')
)

fig_combine.show()

In [34]:
# Line plot for death rate cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, (df_deaths['Mainland China']/df_confirmed['Mainland China']*100).max(), 0.5))

# Create empty figure canvas
fig_rate = go.Figure()
# Add trace to the figure
fig_rate.add_trace(go.Scatter(x=df_deaths['Date'], y=df_deaths['Mainland China']/df_confirmed['Mainland China']*100,
                                mode='lines+markers',
                                line_shape='spline',
                                name='Mainland China Death Rate',
                                line=dict(color='#626262', width=3),
                                marker=dict(size=6, color='#f4f4f2',
                                            line=dict(width=1,color='#626262')),
                                text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_deaths['Date']],
                                hovertext=['Mainland China death rate<br>{:.2f}%'.format(i) for i in df_deaths['Mainland China']/df_confirmed['Mainland China']*100],
                                hovertemplate='<b>%{text}</b><br></br>'+
                                              '%{hovertext}'+
                                              '<extra></extra>'))
fig_rate.add_trace(go.Scatter(x=df_deaths['Date'], y=df_deaths['Other locations']/df_confirmed['Other locations']*100,
                                mode='lines+markers',
                                line_shape='spline',
                                name='Other region Death Rate',
                                line=dict(color='#a7a7a7', width=3),
                                marker=dict(size=6, color='#f4f4f2',
                                            line=dict(width=1,color='#a7a7a7')),
                                text=[datetime.strftime(d, '%b %d %Y AEDT') for d in df_deaths['Date']],
                                hovertext=['Mainland China death rate<br>{:.2f}%'.format(i) for i in df_deaths['Other locations']/df_confirmed['Other locations']*100],
                                hovertemplate='<b>%{text}</b><br></br>'+
                                              '%{hovertext}'+
                                              '<extra></extra>'))

# Customise layout
fig_rate.update_layout(
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=5,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.1f}'.format(i) for i in tickList]
    ),
#    yaxis_title="Total Death Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='rgba(203, 210, 211,.3)',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
#    legend=dict(x=.35, y=-.05),
    plot_bgcolor='#f4f4f2',
    paper_bgcolor='#cbd2d3',
    font=dict(color='#292929')
)

fig_rate.show()

In [22]:
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

# Generate a list for hover text display
textList=[]
for area, region in zip(dfs[keyList[0]]['Province/State'], dfs[keyList[0]]['Country/Region']):
    
    if type(area) is str:
        if region == "Hong Kong" or region == "Macau" or region == "Taiwan":
            textList.append(area)
        else:
            textList.append(area+', '+region)
    else:
        textList.append(region)

# Generate a list for color gradient display
colorList=[]

for comfirmed, recovered, deaths in zip(dfs[keyList[0]]['Confirmed'],dfs[keyList[0]]['Recovered'],dfs[keyList[0]]['Deaths']):
    remaining = recovered / (comfirmed - deaths)
    colorList.append(remaining)
        
fig2 = go.Figure(go.Scattermapbox(
        lat=dfs[keyList[0]]['lat'],
        lon=dfs[keyList[0]]['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color=['#d7191c' if i < 1 else '#1a9622' for i in colorList],
            #colorscale="RdYlGn",
            #colorbar={'thickness':10,'outlinewidth':0,'len':.2},
            size=[math.sqrt(i) for i in dfs[keyList[0]]['Confirmed']], 
            sizemin=1,
            sizemode='area',
            sizeref=2.*max([math.sqrt(i) for i in dfs[keyList[0]]['Confirmed']])/(100.**2),
        ),
        text=textList,
        hovertext=['Comfirmed: {}<br>Recovered: {}<br>Death: {}'.format(i, j, k) for i, j, k in zip(dfs[keyList[0]]['Confirmed'],
                                                                                                    dfs[keyList[0]]['Recovered'],
                                                                                                    dfs[keyList[0]]['Deaths'])],
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "%{hovertext}<br>" +
                        "<extra></extra>")
    
        )

fig2.update_layout(
#    title=dict(
#        text="<b>Latest Coronavirus Outbreak Map<b>",
#        y=0.96, x=0.5, xanchor='center', yanchor='top',
#        font=dict(size=20, color="#292929", family="Playfair Display")
#    ),
    plot_bgcolor='#151920',
    paper_bgcolor='#cbd2d3',
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=0,
        pad=40
    ),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        style="light",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=31.1517252, 
            lon=112.8783222
        ),
        pitch=0,
        zoom=4
    )
)

fig2.show()

In [23]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [ ]:
# This is the version with app.callback

app = dash.Dash(__name__, 
                assets_folder='./assets/',
                meta_tags=[
                    {"name": "author", "content": "Jun Ye"},
                    {"name": "description", "content": "The coronavirus COVID-19 monitor provides up-to-date data for the global spread of coronavirus."},
                    {"property":"og:image", "content": "https://github.com/Perishleaf/data-visualisation-scripts/blob/master/dash-2019-coronavirus/app_screenshot.gif"},
                    {"property":"og:description", "content": "The coronavirus COVID-19 monitor provides up-to-date data for the global spread of coronavirus."},
                    {"property": "og:title", "content": "Coronavirus COVID-19 Outbreak Global Cases Monitor"},
                    {"name": "twitter:title", "content": "Coronavirus COVID-19 Outbreak Global Cases Monitor"},
                    {"name": "viewport", "content": "width=device-width, height=device-height, initial-scale=1.0"}
                ]
      )

app.layout = html.Div(style={'backgroundColor':'#f4f4f2'},
    children=[
        html.Div(
            id="header",
            children=[                          
                html.H4(children="Coronavirus (2019-nCoV) Outbreak Global Cases Monitor"),
                html.P(
                    id="description",
                    children="On Dec 31, 2019, the World Health Organization (WHO) was informed of \
                    an outbreak of “pneumonia of unknown cause” detected in Wuhan City, Hubei Province, China – the \
                    seventh-largest city in China with 11 million residents. As of {}, there are over {:,d} cases \
                    of 2019-nCoV confirmed globally.\
                    This dash board is developed to visualise and track the recent reported \
                    cases on a daily timescale.".format(latestDate, confirmedCases),
                ),
                html.P(style={'fontWeight':'bold'},
                    children="Last updated on {}.".format(latestDate))
            ]        
        ),
        html.Div(
            id="number-plate",
            style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.5%'},
                 children=[
                     html.Div(
                         style={'width':'24.4%','backgroundColor':'#cbd2d3','display':'inline-block',
                                'marginRight':'.8%','verticalAlign':'top'},
                              children=[
                                  html.H3(style={'textAlign':'center',
                                                 'fontWeight':'bold','color':'#2674f6'},
                                               children=[
                                                   html.P(style={'color':'#cbd2d3','padding':'.5rem'},
                                                              children='xxxx xxxx xxxx xxx xxxxx'),
                                                   '{}'.format(daysOutbreak),
                                               ]),
                                  html.H5(style={'textAlign':'center','color':'#2674f6','padding':'.1rem'},
                                               children="Days Since Outbreak")                                        
                                       ]),
                     html.Div(
                         style={'width':'24.4%','backgroundColor':'#cbd2d3','display':'inline-block',
                                'marginRight':'.8%','verticalAlign':'top'},
                              children=[
                                  html.H3(style={'textAlign':'center',
                                                 'fontWeight':'bold','color':'#d7191c'},
                                                children=[
                                                    html.P(style={'padding':'.5rem'},
                                                              children='+ {:,d} from past 24h ({:.1%})'.format(plusConfirmedNum, plusPercentNum1)),
                                                    '{:,d}'.format(confirmedCases)
                                                         ]),
                                  html.H5(style={'textAlign':'center','color':'#d7191c','padding':'.1rem'},
                                               children="Confirmed Cases")                                        
                                       ]),
                     html.Div(
                         style={'width':'24.4%','backgroundColor':'#cbd2d3','display':'inline-block',
                                'marginRight':'.8%','verticalAlign':'top'},
                              children=[
                                  html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#1a9622'},
                                               children=[
                                                   html.P(style={'padding':'.5rem'},
                                                              children='+ {:,d} from past 24h ({:.1%})'.format(plusRecoveredNum, plusPercentNum2)),
                                                   '{:,d}'.format(recoveredCases),
                                               ]),
                                  html.H5(style={'textAlign':'center','color':'#1a9622','padding':'.1rem'},
                                               children="Recovered Cases")                                        
                                       ]),
                     html.Div(
                         style={'width':'24.4%','backgroundColor':'#cbd2d3','display':'inline-block',
                                'verticalAlign':'top'},
                              children=[
                                  html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#6c6c6c'},
                                                children=[
                                                    html.P(style={'padding':'.5rem'},
                                                              children='+ {:,d} from past 24h ({:.1%})'.format(plusDeathNum, plusPercentNum3)),
                                                    '{:,d}'.format(deathsCases)
                                                ]),
                                  html.H5(style={'textAlign':'center','color':'#6c6c6c','padding':'.1rem'},
                                               children="Death Cases")                                        
                                       ])
                          ]),
        html.Div(
            id='dcc-plot',
            style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.35%','marginTop':'.5%'},
                 children=[
                     html.Div(
                         style={'width':'32.79%','display':'inline-block','marginRight':'.8%','verticalAlign':'top'},
                              children=[
                                  html.H5(style={'textAlign':'center','backgroundColor':'#cbd2d3',
                                                 'color':'#292929','padding':'1rem','marginBottom':'0'},
                                               children='Confirmed Case Timeline'),
                                  dcc.Graph(style={'height':'300px'},figure=fig_confirmed)]),
                     html.Div(
                         style={'width':'32.79%','display':'inline-block','marginRight':'.8%','verticalAlign':'top'},
                              children=[
                                  html.H5(style={'textAlign':'center','backgroundColor':'#cbd2d3',
                                                 'color':'#292929','padding':'1rem','marginBottom':'0'},
                                               children='Recovered/Death Case Timeline'),
                                  dcc.Graph(style={'height':'300px'},figure=fig_combine)]),
                     html.Div(
                         style={'width':'32.79%','display':'inline-block','verticalAlign':'top'},
                              children=[
                                  html.H5(style={'textAlign':'center','backgroundColor':'#cbd2d3',
                                                 'color':'#292929','padding':'1rem','marginBottom':'0'},
                                               children='Death Rate Timeline'),
                                  dcc.Graph(style={'height':'300px'},figure=fig_rate)])]),
        html.Div(
            id='dcc-map',
            style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.5%'},
                 children=[
                     html.Div(style={'width':'66.41%','marginRight':'.8%','display':'inline-block','verticalAlign':'top'},
                              children=[
                                  html.H5(style={'textAlign':'center','backgroundColor':'#cbd2d3',
                                                 'color':'#292929','padding':'1rem','marginBottom':'0'},
                                               children='Latest Coronavirus Outbreak Map'),
                                  dcc.Graph(
                                      id='datatable-interact-map',
                                      style={'height':'500px'},
                                  )
                              ]),
                     html.Div(style={'width':'32.79%','display':'inline-block','verticalAlign':'top'},
                              children=[
                                  html.H5(style={'textAlign':'center','backgroundColor':'#cbd2d3',
                                                 'color':'#292929','padding':'1rem','marginBottom':'0'},
                                               children='Cases by Country/Regions'),
                                  dash_table.DataTable(
                                      id='datatable-interact-location',
                                      # Don't show coordinates
                                      columns=[{"name": i, "id": i} for i in dfSum.columns[0:5]],
                                      # But still store coordinates in the table for interactivity
                                      data=dfSum.to_dict("rows"),
                                      row_selectable="single",
                                      #selected_rows=[],
                                      sort_action="native",
                                      style_as_list_view=True,
                                      style_cell={
                                          'font_family':'Arial',
                                          'font_size':'1.2rem',
                                          'padding':'.1rem',
                                          'backgroundColor':'#f4f4f2',
                                      },
                                      fixed_rows={'headers':True,'data':0},
                                      style_table={
                                          'maxHeight':'500px',
                                          #'overflowY':'scroll',
                                          'overflowX':'scroll',
                                      },
                                      style_header={
                                        'backgroundColor':'#f4f4f2',
                                        'fontWeight':'bold'},
                                      style_cell_conditional=[
                                          {'if': {'column_id':'Country/Regions'},'width':'28%'},
                                          {'if': {'column_id':'Remaining'},'width':'18%'},
                                          {'if': {'column_id':'Confirmed'},'width':'18%'},
                                          {'if': {'column_id':'Recovered'},'width':'18%'},
                                          {'if': {'column_id':'Deaths'},'width':'18%'},
                                          {'if': {'column_id':'Confirmed'},'color':'#d7191c'},
                                          {'if': {'column_id':'Recovered'},'color':'#1a9622'},
                                          {'if': {'column_id':'Deaths'},'color':'#6c6c6c'},
                                          {'textAlign': 'center'}
                                      ],
                                  )
                              ])
                 ]),
        html.Div(style={'marginLeft':'1.5%','marginRight':'1.5%'},
                 children=[
                     html.P(style={'textAlign':'center','margin':'auto'},
                            children=["Data source from ",
                                      html.A('Dingxiangyuan, ', href='https://ncov.dxy.cn/ncovh5/view/pneumonia?sce\
                                      ne=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0'),
                                      html.A('Tencent News, ', href='https://news.qq.com//zt2020/page/feiyan.htm#charts'),
                                      'and ', 
                                      html.A('JHU CSSE', href='https://docs.google.com/spreadsheets/d/1yZv9w9z\
                                      RKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#'),
                                      " | 🙏 Pray for China, Pray for the World 🙏 |",
                                      " Developed by ",html.A('Jun', href='https://junye0798.com/')," with ❤️"])])

            ])

@app.callback(
    Output('datatable-interact-map', 'figure'),
    [Input('datatable-interact-location', 'derived_virtual_selected_rows')]
)

def update_figures(derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncracy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []
        
    dff = dfSum
        
    mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

    # Generate a list for hover text display
    textList=[]
    for area, region in zip(dfs[keyList[0]]['Province/State'], dfs[keyList[0]]['Country/Region']):
        
        if type(area) is str:
            if region == "Hong Kong" or region == "Macau" or region == "Taiwan":
                textList.append(area)
            else:
                textList.append(area+', '+region)
        else:
            textList.append(region)
            
    # Generate a list for color gradient display
    colorList=[]

    for comfirmed, recovered, deaths in zip(dfs[keyList[0]]['Confirmed'],dfs[keyList[0]]['Recovered'],dfs[keyList[0]]['Deaths']):
        remaining = recovered / (comfirmed - deaths)
        colorList.append(remaining)

    fig2 = go.Figure(go.Scattermapbox(
        lat=dfs[keyList[0]]['lat'],
        lon=dfs[keyList[0]]['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color=['#d7191c' if i < 1 else '#1a9622' for i in colorList],
            size=[math.sqrt(i) for i in dfs[keyList[0]]['Confirmed']], 
            sizemin=1,
            sizemode='area',
            sizeref=2.*max([math.sqrt(i) for i in dfs[keyList[0]]['Confirmed']])/(100.**2),
        ),
        text=textList,
        hovertext=['Comfirmed: {}<br>Recovered: {}<br>Death: {}'.format(i, j, k) for i, j, k in zip(dfs[keyList[0]]['Confirmed'],
                                                                                                    dfs[keyList[0]]['Recovered'],
                                                                                                    dfs[keyList[0]]['Deaths'])],
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "%{hovertext}<br>" +
                        "<extra></extra>")
    )
    fig2.update_layout(
        plot_bgcolor='#151920',
        paper_bgcolor='#cbd2d3',
        margin=go.layout.Margin(l=10,r=10,b=10,t=0,pad=40),
        hovermode='closest',
        transition = {'duration':1000},
        mapbox=go.layout.Mapbox(
            accesstoken=mapbox_access_token,
            style="light",
            # The direction you're facing, measured clockwise as an angle from true north on a compass
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=3.684188 if len(derived_virtual_selected_rows)==0 else dff['lat'][derived_virtual_selected_rows[0]], 
                lon=148.374024 if len(derived_virtual_selected_rows)==0 else dff['lon'][derived_virtual_selected_rows[0]]
            ),
            pitch=0,
            zoom=1.2 if len(derived_virtual_selected_rows)==0 else 4
        )
    )

    return fig2

if __name__ == '__main__':
    app.run_server(port=8882)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8882/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.8.2&m=1580284437 HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 22:05:58] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1